In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

ss = SparkSession.builder.config('spark.driver.extraClassPath',
                                 'postgresql-42.2.18.jar')\
                         .getOrCreate()
sc = ss.sparkContext

In [3]:
endpoint = 'msds694.cmxsootjz10m.us-west-2.rds.amazonaws.com'
database = 'postgres'
table = 'business'
properties = {'user': 'students', 'password': 'msdsstudents'}
url = 'jdbc:postgresql://%s/%s' % (endpoint, database)

## Create a dataframe from table at the url.

In [4]:
business_df = ss.read.jdbc(url=url, table=table, properties=properties).repartition('zip', 'street').cache()

## Return the first 20 zip and street with the most business. - make sure that the street names are returned fully

In [5]:
business_count_group = business_df.groupBy('zip','street')\
                                  .count()\
                                  .orderBy('count', 'zip',ascending=[False,True])
business_count_group.show(truncate=False)

+-----+----------------------+-----+
|zip  |street                |count|
+-----+----------------------+-----+
|94105|201 Spear St 1100     |118  |
|94104|235 Montgomery St 350 |106  |
|94132|3251 20th Ave         |100  |
|94111|50 California St 1500 |96   |
|94105|156 2nd St            |94   |
|94010|861 Hinckley Rd       |91   |
|94123|1757 Union St         |77   |
|94103|1455 Market St        |74   |
|94114|4248 23rd St          |69   |
|94111|100 Pine St 1250      |66   |
|94117|1365 Grove St         |64   |
|94110|2948 Folsom St        |62   |
|94105|71 Stevenson St 400   |60   |
|94103|123 10th St           |59   |
|94102|588 Sutter St #551    |58   |
|94102|25 Taylor St          |58   |
|10023|1995 Broadway         |55   |
|94103|865 Market St         |55   |
|94102|210 Fell St #2        |52   |
|94111|101 California St 2450|52   |
+-----+----------------------+-----+
only showing top 20 rows



In [7]:
def business_count(x):
    if(x >= 70):
        return 'High'
    elif (x >= 40):
        return 'Medium'
    else:
        return 'low'

business_count_udf = udf(business_count)

## From Q1, add business_count which  returns 'High' when the count >= 70, 'Medium' when the count >= 40, otherwise 'low'

In [8]:
business_count_group.withColumn('business_count', business_count_udf('count')).show()

+-----+--------------------+-----+--------------+
|  zip|              street|count|business_count|
+-----+--------------------+-----+--------------+
|94105|   201 Spear St 1100|  118|          High|
|94104|235 Montgomery St...|  106|          High|
|94132|       3251 20th Ave|  100|          High|
|94111|50 California St ...|   96|          High|
|94105|          156 2nd St|   94|          High|
|94010|     861 Hinckley Rd|   91|          High|
|94123|       1757 Union St|   77|          High|
|94103|      1455 Market St|   74|          High|
|94114|        4248 23rd St|   69|        Medium|
|94111|    100 Pine St 1250|   66|        Medium|
|94117|       1365 Grove St|   64|        Medium|
|94110|      2948 Folsom St|   62|        Medium|
|94105| 71 Stevenson St 400|   60|        Medium|
|94103|         123 10th St|   59|        Medium|
|94102|        25 Taylor St|   58|        Medium|
|94102|  588 Sutter St #551|   58|        Medium|
|10023|       1995 Broadway|   55|        Medium|


In [9]:
ss.stop()